# Тематическое моделирование



In [1]:
# 1
import pandas as pd

import yaml
import pathlib
ROOT = pathlib.Path().resolve().parent.parent

with open(ROOT / "config.yml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

DATA_PATH = ROOT / config['api']['DATA_PATH']

In [2]:
# 2

cm_df = pd.read_csv(DATA_PATH / 'comment' / 'comments_text.csv', sep='\t')

cm_df


,id,snippet.videoId,snippet.textDisplay,snippet.textOriginal,snippet.authorDisplayName,snippet.parentId,cleaned_text,lemmatized_text
0,UgyKlKuQTfZlkw5sqeN4AaABAg,AAjsSrjZXsw,"Люди тупые, они не понимают, что оригинал стои...","Люди тупые, они не понимают, что оригинал стои...",Елена Марьина,-1,"Люди тупые, они не понимают, что оригинал стои...",человек тупой понимать оригинал стоять офигенн...
1,UgxQi90tx9SVF0IauDN4AaABAg,AAjsSrjZXsw,Не понимаю маниакального желания покупать коре...,Не понимаю маниакального желания покупать коре...,Glapfira Nikolaevna,-1,Не понимаю маниакального желания покупать коре...,понимать маниакальный желание покупать корейск...
2,UgwYIO4Dwr8lwOl5_jl4AaABAg,AAjsSrjZXsw,Пару лет назад на Озон купила духи D&G The One...,Пару лет назад на Озон купила духи D&G The One...,Гурия Мирзаева,-1,Пару лет назад на Озон купила духи D&G The One...,пара год назад озон купить дух d g the one жен...
3,UgxvyL4Z2XXYCvnraL54AaABAg,AAjsSrjZXsw,"Я дико извиняюсь, но слово контрофакт, если я ...","Я дико извиняюсь, но слово контрофакт, если я ...",thefallensun_,-1,"Я дико извиняюсь, но слово контрофакт, если я ...",дико извиняться слово контрофакт ошибаться выл...
4,Ugwwha8cCeWK74_460h4AaABAg,AAjsSrjZXsw,Вайлдбериз вообще никого и ничего не уважает и...,Вайлдбериз вообще никого и ничего не уважает и...,Наталья Пирогова,-1,Вайлдбериз вообще никого и ничего не уважает и...,вайлдбериз вообще никто ничто уважать далеко э...
...,...,...,...,...,...,...,...,...
41806,Ugwz6GshlUZMMumE9n54AaABAg.9kO7ERtCXa_9kODjmNACUy,boB2LEXG_zA,Марципану можно придать любую форму-...батончи...,Марципану можно придать любую форму-...батончи...,Nadeschda Schmunk,Ugwz6GshlUZMMumE9n54AaABAg,Марципану можно придать любую форму-...батончи...,марципан придавать любой форма батончик шарик ...
41807,Ugwz6GshlUZMMumE9n54AaABAg.9kO7ERtCXa_9kOAgUwU2Jj,boB2LEXG_zA,@Elena CamilleriУ вас итальянская фамилия...в ...,@Elena CamilleriУ вас итальянская фамилия...в ...,Nadeschda Schmunk,Ugwz6GshlUZMMumE9n54AaABAg,CamilleriУ вас итальянская фамилия...в Италии...,camilleriу итальянский фамилия италия марципан...
41808,Ugwz6GshlUZMMumE9n54AaABAg.9kO7ERtCXa_9kOABWk8eEa,boB2LEXG_zA,и вообще невкусный... я только батончики марци...,и вообще невкусный... я только батончики марци...,Марина Марина,Ugwz6GshlUZMMumE9n54AaABAg,и вообще невкусный... я только батончики марци...,вообще невкусный батончик марципановый шоколад...
41809,Ugwz6GshlUZMMumE9n54AaABAg.9kO7ERtCXa_9kO9mW3prUk,boB2LEXG_zA,А у нас это каштаны называют и кулинарные изде...,А у нас это каштаны называют и кулинарные изде...,Elena Camilleri,Ugwz6GshlUZMMumE9n54AaABAg,А у нас это каштаны называют и кулинарные изде...,это каштан называть кулинарный изделие миндаль...


## Zero-Shot Classification

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()


In [4]:
import numpy as np

t = np.random.choice(cm_df.cleaned_text.values, size=2)
print("Текст: ", t, '\n')

with torch.inference_mode():
    out = model(**tokenizer(t[0], t[1], return_tensors='pt').to(model.device))
    proba = torch.softmax(out.logits, -1).cpu().numpy()[0]

print({v: proba[k] for k, v in model.config.id2label.items()})

Текст:  ['Марципаны это у нас называют каштаны и кулинарные блюда из миндальной муки. Они продаются глазированные в коробках . Ну а что жрал рыжий я не знаю)))'
 'Ну она вообще-то молодая женщина плюс генетика, азиатские женщины долго остаются моложавыми'] 

{'entailment': 0.0039037655, 'contradiction': 0.11682199, 'neutral': 0.87927425}


In [5]:
def predict_zero_shot(text, label_texts, model, tokenizer, label='entailment', normalize=True):
    label_texts
    tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return proba

def get_prediction_labels(probas, classes):
    res_dict = {}
    for i in range(len(classes)):
        res_dict[classes[i]] = probas[i]

    return res_dict
    #sorted(results, key= lambda x: x[1], reverse=True)

classes = [
    "Продажа",
    "Желание",
    "Бренд",
    "Просмотр",
    "Сомнение",
    "Реклама",
    "Положительный отзыв",
    "Негативный отзыв",
    "Нейтральный отзыв",
]


In [10]:
text = np.random.choice(cm_df.cleaned_text.values, size=1)[0]

res_1 = predict_zero_shot(text, classes, model, tokenizer)
result = get_prediction_labels(res_1, classes)

# Formattings Results
result_flat = list(result.items())
result_flat = sorted(result_flat, key= lambda x: x[1], reverse=True)

print("Text:", text)

print("Classes")
print("=" * 20)
for el in result_flat:
    print(f'{el[0]}\t{el[1]}')

Text: какие красивые упаковки! очень приятно и хочется этой косметикой пользоваться!
Classes
Положительный отзыв	0.313811719417572
Желание	0.23747612535953522
Реклама	0.13859137892723083
Продажа	0.12356521189212799
Просмотр	0.10475553572177887
Бренд	0.041187431663274765
Нейтральный отзыв	0.02754126861691475
Сомнение	0.007787048351019621
Негативный отзыв	0.0052842870354652405


## Old Fashioned